## MODELO FUNCIONAL NO INTERACTIVO

In [1]:
import webview
import threading
from dash import Dash, dcc, html
import dash_table
import pandas as pd
import plotly.express as px

# Aquí incluimos el código de tu dashboard
def create_dash_app():
    # Cargar datos y definir gráficos (tu código de dashboard)
    file_path = "dataframe_api_historico.csv"
    df = pd.read_csv(file_path)
    df_filtered = df.dropna(subset=["Model2"])
    desastres_count = df_filtered[df_filtered["Model2"] != "not informative"]["Model2"].value_counts().reset_index()
    desastres_count.columns = ["Tipo de Desastre", "Total"]

    info_type_filtered = df.dropna(subset=["Model3"])
    info_type_filtered = info_type_filtered[df["Model3"] != "not informative"]
    info_type_count = info_type_filtered["Model3"].value_counts().reset_index()
    info_type_count.columns = ["Categoría de Información", "Total"]

    grouped_bar_model3 = px.bar(
        info_type_count, x='Categoría de Información', y='Total',
        title='Distribución de Tipos de Información',
        labels={'Categoría de Información': 'Categoría de Información', 'Total': 'Cantidad'},
        color='Categoría de Información',
        text_auto=False
    )

    model1_count = df.dropna(subset=["Model1"])["Model1"].value_counts().reset_index()
    model1_count.columns = ["Clasificación", "Total"]

    pie_model1 = px.pie(
        df, names='Model1', title='Distribución de Informative vs Not Related', hole=0.0
    )

    grouped_bar_model2 = px.bar(
        desastres_count, x='Tipo de Desastre', y='Total',
        title='Distribución de Tipos de Desastres', text='Total',
        labels={'Tipo de Desastre': 'Tipo de Desastre', 'Total': 'Cantidad'}
    )

    app = Dash(__name__)
    app.layout = html.Div([
        html.H1("Análisis de Tweets sobre Desastres Naturales", style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
        html.H3(f"Total de Tweets analizados: {df.shape[0]}", style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
        html.Div([
            dcc.Graph(figure=pie_model1, style={'width': '30%', 'display': 'inline-block'}),
            dcc.Graph(figure=grouped_bar_model2, style={'width': '30%', 'display': 'inline-block'}),
            dcc.Graph(figure=grouped_bar_model3, style={'width': '30%', 'display': 'inline-block'}),
        ], style={'textAlign': 'center'}),
        html.Div([
            html.Div([
                html.H3("Conteo de Informative vs Not Related", style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
                dash_table.DataTable(
                    columns=[{"name": col, "id": col} for col in model1_count.columns],
                    data=model1_count.to_dict("records"),
                    page_size=10,
                    style_table={'font-family': 'Arial, sans-serif'}
                ),
            ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'}),
            html.Div([
                html.H3("Conteo de Desastres Naturales", style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
                dash_table.DataTable(
                    columns=[{"name": col, "id": col} for col in desastres_count.columns],
                    data=desastres_count.to_dict("records"),
                    page_size=10,
                    style_table={'font-family': 'Arial, sans-serif'}
                ),
            ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'}),
            html.Div([
                html.H3("Conteo de Tweets por Categoría en Model3", style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
                dash_table.DataTable(
                    columns=[{"name": col, "id": col} for col in info_type_count.columns],
                    data=info_type_count.to_dict("records"),
                    page_size=10,
                    style_table={'font-family': 'Arial, sans-serif'}
                ),
            ], style={'width': '30%', 'display': 'inline-block', 'vertical-align': 'top'}),
        ], style={'textAlign': 'center'}),
        html.H3("Tweets Clasificados", style={'color': 'white', 'font-family': 'Arial, sans-serif'}),
        dash_table.DataTable(
            columns=[{"name": col, "id": col} for col in ["CreatedAt", "Text", "Model1", "Model2", "Model3"]],
            data=df.to_dict("records"),
            page_size=10,
            style_table={'font-family': 'Arial, sans-serif'}
        ),
    ], style={'backgroundColor': '#000000', 'padding': '20px'})
    
    return app

# Start the Dash app on a separate thread
def run_dash():
    app = create_dash_app()
    app.run_server(debug=False, port=8050)

if __name__ == '__main__':
    # Start the Dash app in a thread so that we can open it in a window
    dash_thread = threading.Thread(target=run_dash)
    dash_thread.daemon = True
    dash_thread.start()

    # Open the dashboard in a native window using pywebview
    import webview
    webview.create_window("Dashboard TFM", "http://127.0.0.1:8050", width=1200, height=800)
    webview.start()



C:\Users\branc\AppData\Local\Temp\ipykernel_35684\1147560519.py:4: UserWarning: 
The dash_table package is deprecated. Please replace
`import dash_table` with `from dash import dash_table`

Also, if you're using any of the table format helpers (e.g. Group), replace 
`from dash_table.Format import Group` with 
`from dash.dash_table.Format import Group`
  import dash_table
